# Venture Funding with Deep Learning

A hypothetical venture capital firm Alphabet Soup has provided a CSV file of over 34,000 cases of organizational funding and the outcomes. The request for analysis is attempts at making neural networks which can predict whether an approved application for funding will result in a successful business.

First the data will be prepared for analysis. Then several models of neural networks will be run on the data to see what rate of successful prediction can be achieved. The accuracies of the various models will be displayed and compared. The neural network models will be saved in HDF5 format.

In [1]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `applicants_data.csv` file into a Pandas DataFrame. Review the DataFrame, looking for categorical variables that will need to be encoded, as well as columns that could eventually define your features and target variables.  


In [2]:
# Read the applicants_data.csv file from the Resources folder into a Pandas DataFrame
applicant_data_df = pd.read_csv(Path("./Resources/applicants_data.csv"))

# Review the DataFrame
display(applicant_data_df)


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [196]:
applicant_data_df["IS_SUCCESSFUL"].sum()

18261

In [201]:
applicant_data_df["IS_SUCCESSFUL"].count()

34299

In [202]:
applicant_data_df["IS_SUCCESSFUL"].sum()/applicant_data_df["IS_SUCCESSFUL"].count()

0.5324061926003674

In [3]:
# Review the data types associated with the columns
applicant_data_df.dtypes


EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

### Step 2: Drop the “EIN” (Employer Identification Number) and “NAME” columns from the DataFrame, because they are not relevant to the binary classification model.

In [4]:
# Drop the 'EIN' and 'NAME' columns from the DataFrame
applicant_data_df = applicant_data_df.drop(columns=["EIN", "NAME"])

# Review the DataFrame
applicant_data_df


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


### Step 3: Encode the dataset’s categorical variables using `OneHotEncoder`, and then place the encoded variables into a new DataFrame.

In [5]:
# Create a list of categorical variables 
categorical_variables = list(applicant_data_df.dtypes[applicant_data_df.dtypes=="object"].index)

# Display the categorical variables list
categorical_variables


['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [6]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)


In [7]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(applicant_data_df[categorical_variables])


In [8]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names(categorical_variables))

# Review the DataFrame
encoded_df

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Step 4: Add the original DataFrame’s numerical variables to the DataFrame containing the encoded variables.

> **Note** To complete this step, you will employ the Pandas `concat()` function that was introduced earlier in this course. 

In [9]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
preprocessed_df = pd.concat([applicant_data_df.drop(columns=categorical_variables), encoded_df], axis=1)

# Review the Dataframe
preprocessed_df


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,1,5000,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Step 5: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “IS_SUCCESSFUL”. The remaining columns should define the features dataset. 



In [10]:
# Define the target set y using the IS_SUCCESSFUL column
y = preprocessed_df["IS_SUCCESSFUL"]

# Display a sample of y
display(y)


0        1
1        1
2        0
3        1
4        1
        ..
34294    0
34295    0
34296    0
34297    1
34298    0
Name: IS_SUCCESSFUL, Length: 34299, dtype: int64

In [11]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = preprocessed_df.drop(columns=["IS_SUCCESSFUL"])

# Review the features DataFrame
display(X)

,STATUS,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,1,5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,1,5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,1,5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Step 6: Split the features and target sets into training and testing datasets.


In [12]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


### Step 7: Use scikit-learn's `StandardScaler` to scale the features data.

In [13]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


---

## Compile and Evaluate a Binary Classification Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [14]:
pd.DataFrame(X_train_scaled)

,0,1,2,3,4,5,6,7,8,9,...,106,107,108,109,110,111,112,113,114,115
0,0.0108,-0.029571,-0.123588,-0.029915,-0.045438,-0.008818,-0.006235,-0.006235,-0.178463,-0.020683,...,-0.150669,-0.124397,-0.331791,-0.082045,-0.16762,-0.348999,-0.06402,-0.075291,0.029257,-0.029257
1,0.0108,-0.029571,-0.123588,-0.029915,-0.045438,-0.008818,-0.006235,-0.006235,-0.178463,-0.020683,...,-0.150669,-0.124397,-0.331791,-0.082045,-0.16762,-0.348999,-0.06402,-0.075291,0.029257,-0.029257
2,0.0108,-0.029571,-0.123588,-0.029915,-0.045438,-0.008818,-0.006235,-0.006235,-0.178463,-0.020683,...,-0.150669,-0.124397,-0.331791,-0.082045,-0.16762,-0.348999,-0.06402,-0.075291,0.029257,-0.029257
3,0.0108,-0.029571,-0.123588,-0.029915,-0.045438,-0.008818,-0.006235,-0.006235,-0.178463,-0.020683,...,-0.150669,-0.124397,-0.331791,-0.082045,-0.16762,-0.348999,-0.06402,-0.075291,0.029257,-0.029257
4,0.0108,-0.029571,8.091371,-0.029915,-0.045438,-0.008818,-0.006235,-0.006235,-0.178463,-0.020683,...,-0.150669,-0.124397,-0.331791,-0.082045,-0.16762,-0.348999,-0.06402,-0.075291,0.029257,-0.029257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25719,0.0108,-0.029571,-0.123588,-0.029915,-0.045438,-0.008818,-0.006235,-0.006235,-0.178463,-0.020683,...,-0.150669,-0.124397,-0.331791,-0.082045,-0.16762,-0.348999,-0.06402,-0.075291,0.029257,-0.029257
25720,0.0108,-0.029571,-0.123588,-0.029915,-0.045438,-0.008818,-0.006235,-0.006235,-0.178463,-0.020683,...,-0.150669,-0.124397,-0.331791,-0.082045,-0.16762,-0.348999,-0.06402,-0.075291,0.029257,-0.029257
25721,0.0108,-0.029571,-0.123588,-0.029915,-0.045438,-0.008818,-0.006235,-0.006235,-0.178463,-0.020683,...,-0.150669,-0.124397,-0.331791,-0.082045,-0.16762,-0.348999,-0.06402,-0.075291,0.029257,-0.029257
25722,0.0108,-0.029571,-0.123588,-0.029915,-0.045438,-0.008818,-0.006235,-0.006235,-0.178463,-0.020683,...,-0.150669,-0.124397,-0.331791,-0.082045,-0.16762,-0.348999,-0.06402,-0.075291,0.029257,-0.029257


In [15]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
print(number_input_features)


116


In [16]:
# Define the number of neurons in the output layer
number_output_neurons = 1

In [17]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = (number_input_features + number_output_neurons)//2

# Review the number hidden nodes in the first layer
hidden_nodes_layer1

58

In [18]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = (hidden_nodes_layer1 + number_output_neurons)//2

# Review the number hidden nodes in the second layer
hidden_nodes_layer2

29

In [19]:
# Create the Sequential model instance
nn = Sequential()

In [20]:
# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

In [21]:
# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

In [22]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=1, activation="sigmoid"))

In [23]:
# Display the Sequential model summary
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 58)                6786      
                                                                 
 dense_1 (Dense)             (None, 29)                1711      
                                                                 
 dense_2 (Dense)             (None, 1)                 30        
                                                                 
Total params: 8,527
Trainable params: 8,527
Non-trainable params: 0
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [24]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [25]:
# Fit the model using 50 epochs and the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)


Epoch 1/50
804/804 [==============================] - 1s 569us/step - loss: 0.5748 - accuracy: 0.7170
Epoch 2/50
804/804 [==============================] - 0s 572us/step - loss: 0.5527 - accuracy: 0.7286
Epoch 3/50
804/804 [==============================] - 0s 571us/step - loss: 0.5493 - accuracy: 0.7318
Epoch 4/50
804/804 [==============================] - 0s 570us/step - loss: 0.5474 - accuracy: 0.7316
Epoch 5/50
804/804 [==============================] - 0s 563us/step - loss: 0.5459 - accuracy: 0.7320
Epoch 6/50
804/804 [==============================] - 0s 568us/step - loss: 0.5450 - accuracy: 0.7324
Epoch 7/50
804/804 [==============================] - 0s 554us/step - loss: 0.5446 - accuracy: 0.7346
Epoch 8/50
804/804 [==============================] - 0s 567us/step - loss: 0.5432 - accuracy: 0.7339
Epoch 9/50
804/804 [==============================] - 0s 556us/step - loss: 0.5430 - accuracy: 0.7341
Epoch 10/50
804/804 [==============================] - 0s 564us/step - loss: 0.542

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [26]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5557 - accuracy: 0.7291 - 153ms/epoch - 569us/step
Loss: 0.5557466149330139, Accuracy: 0.7290962338447571


### Step 4: Save and export your model to an HDF5 file, and name the file `AlphabetSoup.h5`. 


In [27]:
# Set the model's file path
file_path = Path("./AlphabetSoup.h5")

# Export your model to a HDF5 file
nn.save(file_path)


---

## Optimize the neural network model


### Step 1: Define at least three new deep neural network models (resulting in the original plus 3 optimization attempts). With each, try to improve on your first model’s predictive accuracy.

> **Rewind** Recall that perfect accuracy has a value of 1, so accuracy improves as its value moves closer to 1. To optimize your model for a predictive accuracy as close to 1 as possible, you can use any or all of the following techniques:
>
> * Adjust the input data by dropping different features columns to ensure that no variables or outliers confuse the model.
>
> * Add more neurons (nodes) to a hidden layer.
>
> * Add more hidden layers.
>
> * Use different activation functions for the hidden layers.
>
> * Add to or reduce the number of epochs in the training regimen.


In [28]:
# Model Ideas:
# More neurons in 2nd layer than 1st layer\
# much smaller amount of neurons in 3 or 4 layers
# remove the special considerations column and status columns
# break funding requests into three sets, one for up to 100k, one for 100k-1M, and one for over 1M
# take the log of the funding requests ASK_AMT

# Implement:
#  Remove STATUS and SPECIAL_CONSIDERATIONS
#  Separate ASK_AMT to several levels to analyze, then apply a log function to it

# Attempt to use alternatively preprocessed data with first neural network setup
# Try again with Nadam optimizer
# Try again with Nadam optimizer and a third layer
# Do 1000 epochs for each






### Alternative Preprocessing 1:

In [29]:
# Preprocessing Alt-1:

In [30]:
# remove STATUS and SPECIAL_CONSIDERATIONS columns, as they appear to have all the same value in them.
applicant_data_df_a1 = applicant_data_df.drop(columns=["STATUS", "SPECIAL_CONSIDERATIONS"])

In [31]:
display(applicant_data_df_a1)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,5000,0
34297,T5,Independent,C3000,ProductDev,Association,0,5000,1


In [32]:
# retrieve max ASK_AMT
applicant_data_df_a1["ASK_AMT"].max()

8597806340

In [33]:
# apply natural log scale to ASK_AMT
applicant_data_df_a1["ASK_AMT"]=np.log(applicant_data_df_a1["ASK_AMT"])

In [34]:
display(applicant_data_df_a1)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,8.517193,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,11.595335,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,8.517193,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,8.808668,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,11.867729,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,8.517193,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,8.517193,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,8.517193,0
34297,T5,Independent,C3000,ProductDev,Association,0,8.517193,1


In [35]:
# find all categories of INCOME_AMT
applicant_data_df_a1["INCOME_AMT"].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [36]:
# relabel each income group by the max value it can represent. for 50M+, will use 10 billion, as the highest value asked for in the data set is in the 8 billion range. Used 1 for 0 so log function will work and return 0.
applicant_data_df_a1["INCOME_AMT"] = applicant_data_df_a1["INCOME_AMT"].replace(["0", "25000-99999", "100000-499999", "1M-5M", "1-9999", "10000-24999", "10M-50M", "5M-10M", "50M+"], [1, 100000, 500000, 5000000, 10000, 25000, 50000000, 10000000, 10000000000])

In [37]:
display(applicant_data_df_a1)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,8.517193,1
1,T3,Independent,C2000,Preservation,Co-operative,10000,11.595335,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,8.517193,0
3,T3,CompanySponsored,C2000,Preservation,Trust,25000,8.808668,1
4,T3,Independent,C1000,Heathcare,Trust,500000,11.867729,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,8.517193,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,8.517193,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,8.517193,0
34297,T5,Independent,C3000,ProductDev,Association,1,8.517193,1


In [38]:
# apply natural log to income amounts
applicant_data_df_a1["INCOME_AMT"]=np.log(applicant_data_df_a1["INCOME_AMT"])

In [39]:
display(applicant_data_df_a1)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0.000000,8.517193,1
1,T3,Independent,C2000,Preservation,Co-operative,9.210340,11.595335,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0.000000,8.517193,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10.126631,8.808668,1
4,T3,Independent,C1000,Heathcare,Trust,13.122363,11.867729,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0.000000,8.517193,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0.000000,8.517193,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0.000000,8.517193,0
34297,T5,Independent,C3000,ProductDev,Association,0.000000,8.517193,1


In [40]:
# recapitulation of preprocessing steps taken above, onehot encoding and standard scaler, but for the data with irrelevant columns removed and natural log for income and ask amounts.



In [41]:
# Create a list of categorical variables 
categorical_variables = list(applicant_data_df_a1.dtypes[applicant_data_df_a1.dtypes=="object"].index)

# Display the categorical variables list
categorical_variables


['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION']

In [42]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)


In [43]:
# Encode the categorcal variables using OneHotEncoder
encoded_data_a1 = enc.fit_transform(applicant_data_df_a1[categorical_variables])


In [44]:
# Create a DataFrame with the encoded variables
encoded_df_a1 = pd.DataFrame(
    encoded_data_a1,
    columns = enc.get_feature_names(categorical_variables))

# Review the DataFrame
encoded_df_a1

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,CLASSIFICATION_C8210,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
34295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
34296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
34297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


### Step 4: Add the original DataFrame’s numerical variables to the DataFrame containing the encoded variables.

> **Note** To complete this step, you will employ the Pandas `concat()` function that was introduced earlier in this course. 

In [45]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
preprocessed_df_a1 = pd.concat([applicant_data_df_a1.drop(columns=categorical_variables), encoded_df_a1], axis=1)

# Review the Dataframe
preprocessed_df_a1


,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,CLASSIFICATION_C8210,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,0.000000,8.517193,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,9.210340,11.595335,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,0.000000,8.517193,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,10.126631,8.808668,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,13.122363,11.867729,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0.000000,8.517193,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
34295,0.000000,8.517193,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
34296,0.000000,8.517193,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
34297,0.000000,8.517193,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


### Step 5: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “IS_SUCCESSFUL”. The remaining columns should define the features dataset. 



In [46]:
# Define the target set y using the IS_SUCCESSFUL column
y = preprocessed_df_a1["IS_SUCCESSFUL"]

# Display a sample of y
display(y)


0        1
1        1
2        0
3        1
4        1
        ..
34294    0
34295    0
34296    0
34297    1
34298    0
Name: IS_SUCCESSFUL, Length: 34299, dtype: int64

In [47]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = preprocessed_df_a1.drop(columns=["IS_SUCCESSFUL"])

# Review the features DataFrame
display(X)

,INCOME_AMT,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,CLASSIFICATION_C8210,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,0.000000,8.517193,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,9.210340,11.595335,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,0.000000,8.517193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,10.126631,8.808668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,13.122363,11.867729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0.000000,8.517193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
34295,0.000000,8.517193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
34296,0.000000,8.517193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
34297,0.000000,8.517193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


### Step 6: Split the features and target sets into training and testing datasets.


In [48]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


### Step 7: Use scikit-learn's `StandardScaler` to scale the features data.

In [49]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [50]:
display(pd.DataFrame(X_train_scaled))

,0,1,2,3,4,5,6,7,8,9,...,95,96,97,98,99,100,101,102,103,104
0,-0.624023,-0.479573,-0.123588,-0.029915,-0.045438,-0.008818,-0.006235,-0.006235,-0.178463,-0.020683,...,-0.006235,-0.105846,-0.066128,-0.0108,0.472261,-0.447443,1.527695,-0.119471,-0.03638,-1.473467
1,-0.624023,-0.479573,-0.123588,-0.029915,-0.045438,-0.008818,-0.006235,-0.006235,-0.178463,-0.020683,...,-0.006235,-0.105846,-0.066128,-0.0108,-2.117473,2.234921,-0.654581,-0.119471,-0.03638,0.678672
2,-0.624023,-0.479573,-0.123588,-0.029915,-0.045438,-0.008818,-0.006235,-0.006235,-0.178463,-0.020683,...,-0.006235,-0.105846,-0.066128,-0.0108,0.472261,-0.447443,-0.654581,-0.119471,-0.03638,0.678672
3,-0.624023,-0.479573,-0.123588,-0.029915,-0.045438,-0.008818,-0.006235,-0.006235,-0.178463,-0.020683,...,-0.006235,-0.105846,-0.066128,-0.0108,0.472261,-0.447443,-0.654581,-0.119471,-0.03638,0.678672
4,-0.624023,-0.479573,8.091371,-0.029915,-0.045438,-0.008818,-0.006235,-0.006235,-0.178463,-0.020683,...,-0.006235,-0.105846,-0.066128,-0.0108,0.472261,-0.447443,1.527695,-0.119471,-0.03638,-1.473467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25719,-0.624023,-0.479573,-0.123588,-0.029915,-0.045438,-0.008818,-0.006235,-0.006235,-0.178463,-0.020683,...,-0.006235,-0.105846,-0.066128,-0.0108,0.472261,-0.447443,-0.654581,-0.119471,-0.03638,0.678672
25720,-0.624023,-0.479573,-0.123588,-0.029915,-0.045438,-0.008818,-0.006235,-0.006235,-0.178463,-0.020683,...,-0.006235,-0.105846,-0.066128,-0.0108,-2.117473,2.234921,1.527695,-0.119471,-0.03638,-1.473467
25721,-0.624023,-0.479573,-0.123588,-0.029915,-0.045438,-0.008818,-0.006235,-0.006235,-0.178463,-0.020683,...,-0.006235,-0.105846,-0.066128,-0.0108,0.472261,-0.447443,1.527695,-0.119471,-0.03638,-1.473467
25722,-0.624023,-0.479573,-0.123588,-0.029915,-0.045438,-0.008818,-0.006235,-0.006235,-0.178463,-0.020683,...,-0.006235,-0.105846,-0.066128,-0.0108,0.472261,-0.447443,1.527695,-0.119471,-0.03638,-1.473467


### Alternative Model 1

This model uses the same neural network structure as the first one, but uses alternatively preprocessed data. Several irrelevant columns were removed, income amounts ranges were replaced with representative values, and income and ask amounts were applied natural log functions before scaling.

In [51]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

105

In [52]:
# Define the number of neurons in the output layer
number_output_neurons_A1 = 1

In [53]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A1 = (number_input_features + number_output_neurons_A1)//2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A1

53

In [54]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A1 = (hidden_nodes_layer1_A1 + number_output_neurons)//2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer2_A1

27

In [55]:
# Create the Sequential model instance
nn_A1 = Sequential()

In [56]:
# First hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer1_A1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer2_A1, activation="relu"))          
          
# Output layer
nn_A1.add(Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_A1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 53)                5618      
                                                                 
 dense_4 (Dense)             (None, 27)                1458      
                                                                 
 dense_5 (Dense)             (None, 1)                 28        
                                                                 
Total params: 7,104
Trainable params: 7,104
Non-trainable params: 0
_________________________________________________________________


In [57]:
# Compile the Sequential model
nn_A1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [58]:
# Fit the model using 50 epochs and the training data
fit_model_A1 = nn_A1.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 560us/step - loss: 0.5732 - accuracy: 0.7210
Epoch 2/50
804/804 [==============================] - 0s 546us/step - loss: 0.5543 - accuracy: 0.7305
Epoch 3/50
804/804 [==============================] - 0s 555us/step - loss: 0.5507 - accuracy: 0.7303
Epoch 4/50
804/804 [==============================] - 0s 547us/step - loss: 0.5490 - accuracy: 0.7318
Epoch 5/50
804/804 [==============================] - 0s 588us/step - loss: 0.5470 - accuracy: 0.7316
Epoch 6/50
804/804 [==============================] - 0s 589us/step - loss: 0.5469 - accuracy: 0.7312
Epoch 7/50
804/804 [==============================] - 0s 604us/step - loss: 0.5456 - accuracy: 0.7329
Epoch 8/50
804/804 [==============================] - 0s 593us/step - loss: 0.5447 - accuracy: 0.7335
Epoch 9/50
804/804 [==============================] - 0s 564us/step - loss: 0.5439 - accuracy: 0.7331
Epoch 10/50
804/804 [==============================] - 0s 554us/step - loss: 0.543

#### Alternative Model 2

In [59]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

105

In [60]:
# Define the number of neurons in the output layer
number_output_neurons_A2 = 1

In [61]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A2 = (number_input_features + number_output_neurons_A2)//2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A2

53

In [62]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A2 = (hidden_nodes_layer1_A2 + number_output_neurons_A2)//2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer2_A2

27

In [63]:
# Create the Sequential model instance
nn_A2 = Sequential()

In [64]:
# First hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer1_A2, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer2_A1, activation="relu"))          
          
# Output layer
nn_A2.add(Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_A2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 53)                5618      
                                                                 
 dense_7 (Dense)             (None, 27)                1458      
                                                                 
 dense_8 (Dense)             (None, 1)                 28        
                                                                 
Total params: 7,104
Trainable params: 7,104
Non-trainable params: 0
_________________________________________________________________


In [65]:
# Compile the model
nn_A2.compile(loss="binary_crossentropy", optimizer="Nadam", metrics=["accuracy"])


In [66]:
# Fit the model
fit_model_A2 = nn_A2.fit(X_train_scaled, y_train, epochs=50)


Epoch 1/50
804/804 [==============================] - 1s 536us/step - loss: 0.5767 - accuracy: 0.7152
Epoch 2/50
804/804 [==============================] - 0s 545us/step - loss: 0.5532 - accuracy: 0.7284
Epoch 3/50
804/804 [==============================] - 0s 550us/step - loss: 0.5503 - accuracy: 0.7298
Epoch 4/50
804/804 [==============================] - 0s 541us/step - loss: 0.5478 - accuracy: 0.7294
Epoch 5/50
804/804 [==============================] - 0s 539us/step - loss: 0.5470 - accuracy: 0.7305
Epoch 6/50
804/804 [==============================] - 0s 537us/step - loss: 0.5461 - accuracy: 0.7327
Epoch 7/50
804/804 [==============================] - 0s 551us/step - loss: 0.5451 - accuracy: 0.7317
Epoch 8/50
804/804 [==============================] - 0s 553us/step - loss: 0.5444 - accuracy: 0.7325
Epoch 9/50
804/804 [==============================] - 0s 563us/step - loss: 0.5435 - accuracy: 0.7337
Epoch 10/50
804/804 [==============================] - 0s 550us/step - loss: 0.543

#### Alternative Model 3

In [67]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

105

In [68]:
# Define the number of neurons in the output layer
number_output_neurons_A3 = 1

In [69]:
# Define the number of hidden nodes for the hidden layers
hidden_nodes_layer1_A3 = 26
hidden_nodes_layer2_A3 = 16
hidden_nodes_layer3_A3 = 6
hidden_nodes_layer4_A3 = 3

In [70]:
# Create the Sequential model instance
nn_A3 = Sequential()

In [71]:
# First hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer1_A3, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer2_A3, activation="relu"))          
   
# Third hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer3_A3, activation="relu"))          
          
# Fourth hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer4_A3, activation="relu"))               
    
# Output layer
nn_A3.add(Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_A3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 26)                2756      
                                                                 
 dense_10 (Dense)            (None, 16)                432       
                                                                 
 dense_11 (Dense)            (None, 6)                 102       
                                                                 
 dense_12 (Dense)            (None, 3)                 21        
                                                                 
 dense_13 (Dense)            (None, 1)                 4         
                                                                 
Total params: 3,315
Trainable params: 3,315
Non-trainable params: 0
_________________________________________________________________


In [72]:
# Compile the model
nn_A3.compile(loss="binary_crossentropy", optimizer="Nadam", metrics=["accuracy"])


In [73]:
# Fit the model
fit_model_A3 = nn_A3.fit(X_train_scaled, y_train, epochs=50)


Epoch 1/50
804/804 [==============================] - 1s 577us/step - loss: 0.6214 - accuracy: 0.6893
Epoch 2/50
804/804 [==============================] - 0s 587us/step - loss: 0.5705 - accuracy: 0.7274
Epoch 3/50
804/804 [==============================] - 0s 591us/step - loss: 0.5599 - accuracy: 0.7290
Epoch 4/50
804/804 [==============================] - 0s 582us/step - loss: 0.5551 - accuracy: 0.7310
Epoch 5/50
804/804 [==============================] - 0s 583us/step - loss: 0.5521 - accuracy: 0.7310
Epoch 6/50
804/804 [==============================] - 0s 611us/step - loss: 0.5507 - accuracy: 0.7324
Epoch 7/50
804/804 [==============================] - 0s 608us/step - loss: 0.5493 - accuracy: 0.7334
Epoch 8/50
804/804 [==============================] - 0s 583us/step - loss: 0.5493 - accuracy: 0.7322
Epoch 9/50
804/804 [==============================] - 0s 618us/step - loss: 0.5485 - accuracy: 0.7332
Epoch 10/50
804/804 [==============================] - 0s 593us/step - loss: 0.547

### Alternative Preprocessing 2:

In [74]:
# Preprocessing Alt-2:

In [75]:
# remove STATUS and SPECIAL_CONSIDERATIONS columns, as they appear to have all the same value in them.
applicant_data_df_a2 = applicant_data_df.drop(columns=["STATUS", "SPECIAL_CONSIDERATIONS"])

In [76]:
applicant_data_df_a2

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,5000,0
34297,T5,Independent,C3000,ProductDev,Association,0,5000,1


In [168]:
# drop all rows with funding requests over $1,000,000
small_funding_df = applicant_data_df_a2.loc[(applicant_data_df_a2["ASK_AMT"] <= 200000)]

In [169]:
small_funding_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1
...,...,...,...,...,...,...,...,...
34293,T3,CompanySponsored,C1000,Preservation,Association,0,5000,1
34294,T4,Independent,C1000,ProductDev,Association,0,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,5000,0


In [170]:
# apply natural log scale to ASK_AMT
small_funding_df["ASK_AMT"]=np.log(small_funding_df["ASK_AMT"])

C:\Users\phods\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [171]:
display(small_funding_df)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,8.517193,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,11.595335,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,8.517193,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,8.808668,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,11.867729,1
...,...,...,...,...,...,...,...,...
34293,T3,CompanySponsored,C1000,Preservation,Association,0,8.517193,1
34294,T4,Independent,C1000,ProductDev,Association,0,8.517193,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,8.517193,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,8.517193,0


In [172]:
# relabel each income group by the max value it can represent. for 50M+, will use 10 billion, as the highest value asked for in the data set is in the 8 billion range. Used 1 for 0 so log function will work and return 0.
small_funding_df["INCOME_AMT"] = small_funding_df["INCOME_AMT"].replace(["0", "25000-99999", "100000-499999", "1M-5M", "1-9999", "10000-24999", "10M-50M", "5M-10M", "50M+"], [1, 100000, 500000, 5000000, 10000, 25000, 50000000, 10000000, 10000000000])

C:\Users\phods\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [173]:
display(small_funding_df)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,8.517193,1
1,T3,Independent,C2000,Preservation,Co-operative,10000,11.595335,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,8.517193,0
3,T3,CompanySponsored,C2000,Preservation,Trust,25000,8.808668,1
4,T3,Independent,C1000,Heathcare,Trust,500000,11.867729,1
...,...,...,...,...,...,...,...,...
34293,T3,CompanySponsored,C1000,Preservation,Association,1,8.517193,1
34294,T4,Independent,C1000,ProductDev,Association,1,8.517193,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,8.517193,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,8.517193,0


In [174]:
# apply natural log to income amounts
small_funding_df["INCOME_AMT"]=np.log(small_funding_df["INCOME_AMT"])

C:\Users\phods\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [175]:
display(small_funding_df)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0.000000,8.517193,1
1,T3,Independent,C2000,Preservation,Co-operative,9.210340,11.595335,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0.000000,8.517193,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10.126631,8.808668,1
4,T3,Independent,C1000,Heathcare,Trust,13.122363,11.867729,1
...,...,...,...,...,...,...,...,...
34293,T3,CompanySponsored,C1000,Preservation,Association,0.000000,8.517193,1
34294,T4,Independent,C1000,ProductDev,Association,0.000000,8.517193,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0.000000,8.517193,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0.000000,8.517193,0


In [176]:
# reset the index of the small funding dataframe
small_funding_df = small_funding_df.reset_index()

In [177]:
# recapitulation of preprocessing steps taken above, onehot encoding and standard scaler, but for the data with irrelevant columns removed and natural log for income and ask amounts.

In [178]:
# Create a list of categorical variables 
categorical_variables = list(small_funding_df.dtypes[small_funding_df.dtypes=="object"].index)

# Display the categorical variables list
categorical_variables


['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION']

In [179]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)


In [180]:
# Encode the categorcal variables using OneHotEncoder
encoded_data_a2 = enc.fit_transform(small_funding_df[categorical_variables])


In [181]:
# Create a DataFrame with the encoded variables
encoded_df_a2 = pd.DataFrame(
    encoded_data_a2,
    columns = enc.get_feature_names(categorical_variables))

# Review the DataFrame
encoded_df_a2

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T15,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,CLASSIFICATION_C8210,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30892,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
30893,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
30894,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
30895,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


### Step 4: Add the original DataFrame’s numerical variables to the DataFrame containing the encoded variables.

> **Note** To complete this step, you will employ the Pandas `concat()` function that was introduced earlier in this course. 

In [182]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
preprocessed_df_a2 = pd.concat([small_funding_df.drop(columns=categorical_variables), encoded_df_a2], axis=1)

# Review the Dataframe
preprocessed_df_a2


,index,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T15,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,CLASSIFICATION_C8210,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,0,0.000000,8.517193,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,1,9.210340,11.595335,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,2,0.000000,8.517193,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,3,10.126631,8.808668,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,4,13.122363,11.867729,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30892,34293,0.000000,8.517193,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
30893,34294,0.000000,8.517193,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
30894,34295,0.000000,8.517193,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
30895,34296,0.000000,8.517193,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


### Step 5: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “IS_SUCCESSFUL”. The remaining columns should define the features dataset. 



In [183]:
# Define the target set y using the IS_SUCCESSFUL column
y = preprocessed_df_a2["IS_SUCCESSFUL"]

# Display a sample of y
display(y)


0        1
1        1
2        0
3        1
4        1
        ..
30892    1
30893    0
30894    0
30895    0
30896    1
Name: IS_SUCCESSFUL, Length: 30897, dtype: int64

In [184]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = preprocessed_df_a2.drop(columns=["IS_SUCCESSFUL"])

# Review the features DataFrame
display(X)

,index,INCOME_AMT,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T15,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T3,...,CLASSIFICATION_C8210,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,0,0.000000,8.517193,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,1,9.210340,11.595335,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,2,0.000000,8.517193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,3,10.126631,8.808668,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,4,13.122363,11.867729,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30892,34293,0.000000,8.517193,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
30893,34294,0.000000,8.517193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
30894,34295,0.000000,8.517193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
30895,34296,0.000000,8.517193,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


### Step 6: Split the features and target sets into training and testing datasets.


In [185]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X2_train, X2_test, y2_train, y2_test = train_test_split(X, y, random_state=1)


### Step 7: Use scikit-learn's `StandardScaler` to scale the features data.

In [186]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X2_scaler = scaler.fit(X2_train)

# Fit the scaler to the features training dataset
X2_train_scaled = scaler.transform(X2_train)
X2_test_scaled = scaler.transform(X2_test)


In [187]:
display(pd.DataFrame(X2_train_scaled))

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-1.009361,-0.514268,-0.417456,-0.128949,-0.024587,-0.044111,-0.006569,-0.186018,-0.014691,0.525482,...,-0.006569,-0.107557,-0.030827,-0.006569,0.468045,-0.448186,-0.682321,-0.094477,-0.032864,0.697994
1,-1.036504,-0.514268,-0.417456,-0.128949,-0.024587,-0.044111,-0.006569,-0.186018,-0.014691,0.525482,...,-0.006569,-0.107557,-0.030827,-0.006569,0.468045,-0.448186,1.465586,-0.094477,-0.032864,-1.432677
2,1.354482,-0.514268,-0.417456,-0.128949,-0.024587,-0.044111,-0.006569,-0.186018,-0.014691,0.525482,...,-0.006569,-0.107557,-0.030827,-0.006569,0.468045,-0.448186,-0.682321,-0.094477,-0.032864,0.697994
3,0.300620,1.378457,-0.417456,-0.128949,-0.024587,-0.044111,-0.006569,-0.186018,-0.014691,-1.903013,...,-0.006569,-0.107557,-0.030827,-0.006569,-2.136549,2.231217,-0.682321,-0.094477,-0.032864,0.697994
4,0.576024,2.182378,2.307249,-0.128949,-0.024587,-0.044111,-0.006569,-0.186018,-0.014691,-1.903013,...,-0.006569,-0.107557,-0.030827,-0.006569,-2.136549,2.231217,-0.682321,-0.094477,-0.032864,0.697994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23167,0.170111,-0.514268,-0.417456,-0.128949,-0.024587,-0.044111,-0.006569,-0.186018,-0.014691,0.525482,...,-0.006569,-0.107557,-0.030827,-0.006569,0.468045,-0.448186,-0.682321,-0.094477,-0.032864,0.697994
23168,-1.150482,-0.514268,-0.417456,-0.128949,-0.024587,-0.044111,-0.006569,-0.186018,-0.014691,0.525482,...,-0.006569,-0.107557,-0.030827,-0.006569,0.468045,-0.448186,1.465586,-0.094477,-0.032864,-1.432677
23169,-0.372942,-0.514268,-0.417456,-0.128949,-0.024587,-0.044111,-0.006569,-0.186018,-0.014691,0.525482,...,-0.006569,-0.107557,-0.030827,-0.006569,0.468045,-0.448186,1.465586,-0.094477,-0.032864,-1.432677
23170,-1.705576,-0.514268,-0.417456,-0.128949,-0.024587,-0.044111,-0.006569,-0.186018,-0.014691,-1.903013,...,-0.006569,-0.107557,-0.030827,-0.006569,-2.136549,2.231217,1.465586,-0.094477,-0.032864,-1.432677


In [188]:
# Define the the number of inputs (features) to the model
number_input_features = len(X2_train.iloc[0])

# Review the number of features
number_input_features

100

In [189]:
# Define the number of neurons in the output layer
number_output_neurons_A4 = 1

In [190]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A4 = (number_input_features + number_output_neurons_A4)//2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A4

50

In [191]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A4 = (hidden_nodes_layer1_A4 + number_output_neurons_A4)//2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer2_A4

25

In [192]:
# Create the Sequential model instance
nn_A4 = Sequential()

In [193]:
# First hidden layer
nn_A4.add(Dense(units=hidden_nodes_layer1_A2, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_A4.add(Dense(units=hidden_nodes_layer2_A1, activation="relu"))          
          
# Output layer
nn_A4.add(Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_A4.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 53)                5353      
                                                                 
 dense_24 (Dense)            (None, 27)                1458      
                                                                 
 dense_25 (Dense)            (None, 1)                 28        
                                                                 
Total params: 6,839
Trainable params: 6,839
Non-trainable params: 0
_________________________________________________________________


In [194]:
# Compile the model
nn_A4.compile(loss="binary_crossentropy", optimizer="Nadam", metrics=["accuracy"])


In [195]:
# Fit the model
fit_model_A4 = nn_A4.fit(X2_train_scaled, y2_train, epochs=50)


Epoch 1/50
725/725 [==============================] - 1s 543us/step - loss: 0.5607 - accuracy: 0.7320
Epoch 2/50
725/725 [==============================] - 0s 521us/step - loss: 0.5375 - accuracy: 0.7454
Epoch 3/50
725/725 [==============================] - 0s 522us/step - loss: 0.5345 - accuracy: 0.7453
Epoch 4/50
725/725 [==============================] - 0s 522us/step - loss: 0.5328 - accuracy: 0.7472
Epoch 5/50
725/725 [==============================] - 0s 561us/step - loss: 0.5310 - accuracy: 0.7472
Epoch 6/50
725/725 [==============================] - 0s 610us/step - loss: 0.5287 - accuracy: 0.7488
Epoch 7/50
725/725 [==============================] - 0s 551us/step - loss: 0.5280 - accuracy: 0.7483
Epoch 8/50
725/725 [==============================] - 0s 543us/step - loss: 0.5269 - accuracy: 0.7495
Epoch 9/50
725/725 [==============================] - 0s 597us/step - loss: 0.5258 - accuracy: 0.7495
Epoch 10/50
725/725 [==============================] - 0s 600us/step - loss: 0.525

### Step 2: After finishing your models, display the accuracy scores achieved by each model, and compare the results.

In [ ]:
print("Original Model Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = # YOUR CODE HERE

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
print("Alternative Model 1 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy =# YOUR CODE HERE

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
print("Alternative Model 2 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = # YOUR CODE HERE

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

### Step 3: Save each of your alternative models as an HDF5 file.


In [ ]:
# Set the file path for the first alternative model
file_path = # YOUR CODE HERE

# Export your model to a HDF5 file
# YOUR CODE HERE


In [ ]:
# Set the file path for the second alternative model
file_path = # YOUR CODE HERE

# Export your model to a HDF5 file
# YOUR CODE HERE
